# Exercise 6.2: Two masses connected by strings. 

Two masses 1 and 2, of  weights $W_1, W_2$, respectively, are hung from three pieces of string with lengths $L_1, L_2, L_3$ and a horizontal bar of length $L$. 

Using $N$-dimensional Newton-Raphson searching, find the angles $\theta_1$, $\theta_2$ 
and $\theta_3$ and the tensions exerted by the strings $T_1$, $T_2$, $T_3$. 

Use the values: $W_1 = 10$ N, $W_2 = 20$ N, $(L_1, L_2, L_3)=(3,4,4)$ m and $L=8$ m. 


## Solution

We begin by implementing a function that approximates the partial derivative of a function. This is achieved using the function ```partiald```, which takes as inputs the function to be differentiated, the point at which the derivative is evaluated, the direction in which we are differentiating, and the step size for a central difference derivative.

In [16]:
import numpy as np
from numpy import linalg

def partiald(func,a,i,h): # approximates the partial derivaitve of func wrt coordinate i at point a with step size h
    
    dx = [] # variation in the ith coordinate
    for xi in range(len(a)): # defines the increment in the function input: a vector with h in the ith coordinate and zeros everywhere else
        if xi == i:
            dx.append(h)
        else:
            dx.append(0)
    
    df = (func(np.array(a)+np.array(dx)/2)-func(np.array(a)-np.array(dx)/2)) # the increment is f(x) as a result of dx
    return df/h # the partial 

def jacobian(func,a,h): # accepts a list func of component functions and returns the Jacobian at a
    derivative = []
    for j in range(len(a)):
        row = []
        for i in range(len(a)):
            row.append(partiald(func[j],a,i,h))
        derivative.append(row)
    return derivative
    

In [55]:
weights = [10,20]
lengths = [3,4,4,8] 

def f0(x):
    return x[6]*x[0]-x[7]*x[1]-weights[0]
def f1(x): 
    return x[6]*x[3]-x[7]*x[4]
def f2(x):
    return x[7]*x[1]+x[8]*x[2]-weights[1]
def f3(x):
    return x[7]*x[4]-x[8]*x[5]
def f4(x):
    return lengths[0]-x[3]+lengths[1]*x[4]+lengths[2]*x[5]-lengths[3]
def f5(x):
    return lengths[0]*x[0]+lengths[1]*x[1]-lengths[2]*x[2]
def f6(x):
    return x[0]**2+x[3]**2
def f7(x):
    return x[1]**2+x[4]**2
def f8(x):
    return x[2]**2+x[5]**2

def f(x):
    return np.array([f0(x),f1(x),f2(x),f3(x),f4(x),f5(x),f6(x),f7(x),f8(x)])


In [56]:
def newton(func,x0,h,prec,N,nUpdate):
    a = x0 # sets the initial guess of the root to x0
    count = 0 # the number of root searches
    
    precCheck = []
    for i in range(len(x0)):
        precCheck.append(0) # fills a list with zeros for a check of the precision
    precGoal = []
    for i in range(len(x0)):
        precGoal.append(1) # loop ends once precCheck is filled with ones
        
    while precCheck != precGoal and count < N:
        dx = linalg.solve[jacobian(func,a,h),func] # the correction to the root
        a = x0 + dx
        
        for i in range(len(x0)): # if the ith value of func is within prec of zero, then the ith entry of precCheck is set to 1
            if func(a)[i] < prec and count%nUpdate == 0:
                precCheck[i] = 1
                
        count = count + 1
        
    return(a)

In [57]:
sqrt2 = 1/np.sqrt(2)
a = np.array([sqrt2,sqrt2,sqrt2,sqrt2,sqrt2,sqrt2,1,1,1])
newton(f,a,1E-5,1E-5,100000,10)

TypeError: 'function' object is not subscriptable